<a href="https://colab.research.google.com/github/chirayu-xx/pdf-summarizer/blob/main/pdf_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install transformers
!pip install pytesseract
!pip install PyMuPDF
!pip install sentencepiece

In [ ]:
 !pip install colabcode

In [ ]:
!pip install fastapi==0.103.2 nest-asyncio pyngrok uvicorn
!pip install python-multipart

In [4]:
!pip install sentencepiece
!ngrok authtoken 2ZLDE06RzMLQEbRfbRmZzhHTdSJ_5vnBBVzitpreaSLRbFGxD

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI, File, UploadFile,HTTPException
from fastapi.responses import JSONResponse
from typing import List
import os
import fitz  # PyMuPDF library
from transformers import AutoTokenizer, AutoModelWithLMHead

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Path to store uploaded files
upload_directory = '/content/uploaded_files'
text_directory = '/content/text_files'


# Home route
@app.get("/")
def read_root():
    return {"message": "Welcome to the FastAPI server!"}


@app.post("/file")
async def upload_file(file: UploadFile = File(...)):
    # Do here your stuff with the file
    return {"filename": file.filename}

@app.post("/extract-text")
async def extract_text_from_pdf(file: UploadFile = UploadFile(...)):
    try:
        # Read PDF content
        pdf_content = await file.read()

        # Open PDF with PyMuPDF
        pdf_document = fitz.open("pdf", pdf_content)

        # Initialize an empty string to store extracted text
        text = ""

        # Iterate through pages and extract text
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            text += page.get_text()

        # Close the PDF document
        pdf_document.close()

        return JSONResponse(content={"extracted_text": text}, status_code=200)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/summarize")
async def extract_text_from_pdf(file: UploadFile = UploadFile(...)):
    try:
        # Read PDF content
        pdf_content = await file.read()

        # Open PDF with PyMuPDF
        pdf_document = fitz.open("pdf", pdf_content)

        # Initialize an empty string to store extracted text
        text = ""

        # Iterate through pages and extract text
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            text += page.get_text()

        # Initialize the model and tokenizer
        tokenizer=AutoTokenizer.from_pretrained('T5-base')
        model=AutoModelWithLMHead.from_pretrained('T5-base', return_dict=True)

        # Encode the text
        inputs = tokenizer.encode("summarize: " + text, return_tensors="pt",truncation=True)

        # Generate the summary
        outputs = model.generate(inputs,
        max_length=1000, min_length=100)

        # Decode the summary
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return JSONResponse(content={"summary": summary}, status_code=200)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Example Usage:
# Send a PDF file to http://your-fastapi-host/extract-text using your frontend.


In [ ]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)
server.run_app(app=app)

INFO:     Started server process [470]
INFO:uvicorn.error:Started server process [470]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://b9d5-34-106-233-51.ngrok-free.app" -> "http://localhost:10000"
